# Market Simulator
ML for trading Udacity Course exercise

More info:
http://wiki.quantsoftware.org/index.php?title=CompInvesti_Homework_3

A transcription of the Udacity Course lectures can be find on https://docs.google.com/document/d/1ELqlnuTSdc9-MDHOkV0uvSY4RmI1eslyQlU9DgOY_jc/edit?usp=sharing

Kairoart 2018
"""


## Overview

In this project you will create a basic market simulator that accepts trading orders and keeps track of a portfolio's value and saves it to a file. You will also create another program that assesses the performance of that portfolio. 

## Part 1: Create a market simulation tool

Starting cash: 1000000  
Input file: orders.csv 
Output file: values.csv

The file of orders is organized like this:

    Year
    Month
    Day
    Symbol
    BUY or SELL
    Number of Shares 
    
For example:

    2008, 12, 3, AAPL, BUY, 130
    2008, 12, 8, AAPL, SELL, 130
    2008, 12, 5, IBM, BUY, 50


### Goal

Your simulator should calculate the total value of the portfolio for each day using adjusted closing prices (cash plus value of equities) and print the result to the file values.csv. The contents of the values.csv file should look something like this:

    2008, 12, 3, 1000000
    2008, 12, 4, 1000010
    2008, 12, 5, 1000250
    ...


### 1. Import libraries

In [19]:
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import operator
import sys
import csv

# QSTK Imports
import QSTK.qstkutil.qsdateutil as du
import QSTK.qstkutil.tsutil as tsu
import QSTK.qstkutil.DataAccess as da

ModuleNotFoundError: No module named 'QSTK'

In [14]:
def readOrdersFileIntoDF(filename):

    # opening the filename
    fr = open(filename)
    
    # for row count in 
    index=0
    
    # Lists used for making the dataframe.
    dtList = []
    symbolList = []
    orderTypeList = []
    volumeList = []
    
    # For each line
    # A Sample Line - 2011,1,14,AAPL,Buy,1500
    for orderString in fr.readlines():

        # Stripping off the return line character
        orderString=orderString.strip()
        
        # Splitting the line and getting a List back
        listFromLine = orderString.split(',')
        
        # Adding the dates into dtList. 16,00,00 for 1600 hrs
        dtList.append(dt.datetime(int(listFromLine[0]), int(listFromLine[1]), int(listFromLine[2]), 16, 00, 00))
        
        # Adding the symbols into symbolList
        symbolList.append(listFromLine[3])
        
        # Adding the orders into orderTypeList
        orderTypeList.append(listFromLine[4])
        
        # Adding the number of shares into volumeList
        volumeList.append(listFromLine[5])

    # Creating a Dictionary for converting it into DataFrame later
    data = { 'datetime' : dtList, 'symbol' : symbolList, 'ordertype':orderTypeList, 'volume':volumeList }
    
    # Converting the Dictinary into a nice looking Pandas Dataframe
    ordersDataFrame = pd.DataFrame(data)
    
    #Sorting by datetime column #Makes Sense :)
    sortedOrdersDataFrame = ordersDataFrame.sort_index(by=['datetime'])
    sortedOrdersDataFrame = sortedOrdersDataFrame.reset_index(drop=True)
    
    
    # Making the datetime columns as the index and removing it from the table
    # sortedOrdersDataFrame.index = sortedOrdersDataFrame.datetime
    # del sortedOrdersDataFrame['datetime']
    
    # Getting the Symbols from the Orders. This list will be required for fetching the prices
    symbolList = list(set(sortedOrdersDataFrame['symbol']))
    
    # Returning it.
    return sortedOrdersDataFrame, symbolList

In [15]:
def fetchNYSEData(dt_start, dt_end, ls_symbols):

    # The Time of Closing is 1600 hrs 
    dt_timeofday = dt.timedelta(hours=16)
    
    # Get a list of trading days between the start and the end.
    ldt_timestamps = du.getNYSEdays(dt_start, dt_end, dt_timeofday)

    # Creating an object of the dataaccess class with Yahoo as the source.
    c_dataobj = da.DataAccess('Yahoo', cachestalltime=0)

    # Keys to be read from the data, it is good to read everything in one go.
    ls_keys = ['open', 'high', 'low', 'close', 'volume', 'actual_close']

    # Reading the data, now d_data is a dictionary with the keys above.
    # Timestamps and symbols are the ones that were specified before.
    ldf_data = c_dataobj.get_data(ldt_timestamps, ls_symbols, ls_keys)
    d_data = dict(zip(ls_keys, ldf_data))
    
    timestampsForNYSEDays = d_data['close'].index

    # Filling the data for NAN
    for s_key in ls_keys:
        d_data[s_key] = d_data[s_key].fillna(method='ffill')
        d_data[s_key] = d_data[s_key].fillna(method='bfill')
        d_data[s_key] = d_data[s_key].fillna(1.0)

    # Getting the numpy ndarray of close prices.
    na_price = d_data['close'].values

    # returning the closed prices for all the days    
    return na_price, ldt_timestamps

In [16]:
def marketsim(initialCash, ordersDataFrame, symbols):

    # reading the boundary dates
    dt_start = ordersDataFrame.datetime[0]
    dt_end = ordersDataFrame.datetime[len(ordersDataFrame)-1]
    
    # All the adjustedClosingPrices fetched from NYSE within the range and for given symbols
    closingPrices, ldt_timestamps = fetchNYSEData(dt_start, dt_end, symbols)
    
    num_tradingDays = len(ldt_timestamps)
    
    # For Holdings of the share	
    temp = np.zeros((1, len(symbols)))
    holdings = pd.DataFrame(temp, columns = symbols, index = ['holdings'])
    
    #Cash for the days
    temp = np.zeros((num_tradingDays, 1))
    cash = pd.DataFrame(temp, columns = ['cashinhand'])
    
    #Value for the days
    temp = np.zeros((num_tradingDays, 1))
    valueFrame = pd.DataFrame(temp, columns = ['valueOfPortfolio'])

    #Setting the first value to be the initial cash amount.
    cash.cashinhand.ix[0] = initialCash
    
    index = 0
    
    for tradingDayIndex in range(num_tradingDays):
        if tradingDayIndex != 0:
            cash.cashinhand.ix[tradingDayIndex] = cash.cashinhand.ix[tradingDayIndex - 1] 
        else:
            cash.cashinhand.ix[tradingDayIndex] = initialCash

        for tradingOrder in ordersDataFrame.datetime:				
            if tradingOrder == ldt_timestamps[tradingDayIndex]:
                if ordersDataFrame.ordertype.ix[index] == 'Buy':
                    toBuySymbol = ordersDataFrame.symbol.ix[index]
                    toBuy = symbols.index(toBuySymbol)
                    numShares = ordersDataFrame.volume.ix[index]
                    priceForTheDay = closingPrices[tradingDayIndex, toBuy]
                    cash.cashinhand.ix[tradingDayIndex] = cash.cashinhand.ix[tradingDayIndex] - (priceForTheDay * float(numShares))						
                    holdings[toBuySymbol].ix[0] += int(numShares)

                elif ordersDataFrame.ordertype.ix[index] == 'Sell':
                    toSellSymbol = ordersDataFrame.symbol.ix[index]
                    toSell = symbols.index(toSellSymbol)
                    numShares = ordersDataFrame.volume.ix[index]
                    priceForTheDay = closingPrices[tradingDayIndex, toSell]
                    cash.cashinhand.ix[tradingDayIndex] = cash.cashinhand.ix[tradingDayIndex] + (priceForTheDay * float(numShares))						
                    holdings[toSellSymbol].ix[0] -= int(numShares)
                else:
                    print("error")
                index+=1

        valueFromPortfolio = 0

        for symbol in symbols:
            priceForTheDay = closingPrices[tradingDayIndex, symbols.index(symbol)]
            valueFromPortfolio += holdings[symbol].ix[0] * priceForTheDay

        valueFrame.valueOfPortfolio.ix[tradingDayIndex] = valueFromPortfolio + cash.cashinhand.ix[tradingDayIndex]

    valueFrame.index = ldt_timestamps
    return holdings, valueFrame, cash

In [17]:
def writeValuesIntoCSV(valuesFilename, valueFrame):
    file = open(valuesFilename, 'w')
    writer = csv.writer(file)

    for index in range(len(valueFrame)):
        writer.writerow([valueFrame.index[index].year, valueFrame.index[index].month, valueFrame.index[index].day ,int(round(valueFrame.valueOfPortfolio.ix[index], 0))])
    
    file.close()

In [18]:
initialCash = 1000000
ordersFilename = 'input/orders.csv'
valuesFilename = 'output/orders.csv'

# Reading the data from the file, and getting a NumPy matrix
ordersDataFrame, symbols = readOrdersFileIntoDF(ordersFilename)
holdings, valueFrame, cash = marketsim(initialCash, ordersDataFrame, symbols)

writeValuesIntoCSV(valuesFilename, valueFrame)

/home/emi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)


NameError: name 'du' is not defined